`ROLLUP` 有汇总的意思。

`ROLLUP` 与 `CUBE` 的区别：

`CUBE(c1,c2,c3)` 产生这样的集合：
```
(c1, c2, c3)
(c1, c2)
(c2, c3)
(c1,c3)
(c1)
(c2)
(c3)
()
```

`ROLLUP(c1,c2,c3)` 产生这样的集合：
```
(c1, c2, c3)
(c1, c2)
(c1)
()
```

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [2]:
sql = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
    brand VARCHAR NOT NULL,
    segment VARCHAR NOT NULL,
    quantity INT NOT NULL,
    PRIMARY KEY (brand, segment)
);

INSERT INTO sales (brand, segment, quantity)
VALUES
    ('ABC', 'Premium', 100),
    ('ABC', 'Basic', 200),
    ('XYZ', 'Premium', 100),
    ('XYZ', 'Basic', 300);
"""

cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a9db115a60>

In [3]:
sql = """
SELECT
    brand,
    segment,
    SUM (quantity)
FROM
    sales
GROUP BY
    ROLLUP (brand, segment)
ORDER BY
    brand,
    segment;
"""

run_sql(cursor, sql)

  brand  segment  sum
0   ABC    Basic  200
1   ABC  Premium  100
2   ABC     None  300
3   XYZ    Basic  300
4   XYZ  Premium  100
5   XYZ     None  400
6  None     None  700


In [4]:
sql = """
SELECT
    segment,
    brand,
    SUM (quantity)
FROM
    sales
GROUP BY
    ROLLUP (segment, brand)
ORDER BY
    segment,
    brand;
"""

run_sql(cursor, sql)

   segment brand  sum
0    Basic   ABC  200
1    Basic   XYZ  300
2    Basic  None  500
3  Premium   ABC  100
4  Premium   XYZ  100
5  Premium  None  200
6     None  None  700


In [5]:
# partial roll-up
sql = """
SELECT
    segment,
    brand,
    SUM (quantity)
FROM
    sales
GROUP BY
    segment,
    ROLLUP (brand)
ORDER BY
    segment,
    brand;
"""

run_sql(cursor, sql)

   segment brand  sum
0    Basic   ABC  200
1    Basic   XYZ  300
2    Basic  None  500
3  Premium   ABC  100
4  Premium   XYZ  100
5  Premium  None  200


In [6]:
# 使用 DVD 的数据
sql = """
SELECT
    EXTRACT (YEAR FROM rental_date) y,
    EXTRACT (MONTH FROM rental_date) M,
    EXTRACT (DAY FROM rental_date) d,
    COUNT (rental_id)
FROM
    rental
GROUP BY
    ROLLUP (
        EXTRACT (YEAR FROM rental_date),
        EXTRACT (MONTH FROM rental_date),
        EXTRACT (DAY FROM rental_date)
    );
"""

run_sql(cursor, sql)

       y     m     d  count
0   2005     5    24      8
1   2005     5    25    137
2   2005     5    26    174
3   2005     5    27    166
4   2005     5    28    196
5   2005     5    29    154
6   2005     5    30    158
7   2005     5    31    163
8   2005     5  None   1156
9   2005     6    14     16
10  2005     6    15    348
11  2005     6    16    324
12  2005     6    17    325
13  2005     6    18    344
14  2005     6    19    348
15  2005     6    20    331
16  2005     6    21    275
17  2005     6  None   2311
18  2005     7     5     27
19  2005     7     6    504
20  2005     7     7    461
21  2005     7     8    512
22  2005     7     9    513
23  2005     7    10    480
24  2005     7    11    461
25  2005     7    12    495
26  2005     7    26     33
27  2005     7    27    649
28  2005     7    28    620
29  2005     7    29    641
30  2005     7    30    634
31  2005     7    31    679
32  2005     7  None   6709
33  2005     8     1    671
34  2005     8     2